# Installing the environment

## Importing modules and libraries


In [ ]:

from IPython.display import HTML # to display videos
import base64 # to encode videos as base64
from base64 import b64encode # to encode videos as base64
import os # to interact with the operating system
import subprocess # to run commands
import time # to measure execution time
import csv # to save comments
import uuid # to generate unique ids
import cv2 # to split videos
from PIL import Image # to display videos
import pandas as pd # to display comments
import numpy as np # to use Numerical Python
from io import BytesIO #for a binary stream of data in memory


## GitHub

In [ ]:
def download(directory, filename):
    # The base URL of the image files in the GitHub repository
    base_url = 'https://raw.githubusercontent.com/Denis2054/RAG-Driven-Generative-AI/main/'

    # Complete URL for the file
    file_url = f"{base_url}{directory}/{filename}"

    # Use curl to download the file, including an Authorization header for the private token
    try:
        # Prepare the curl command
        curl_command = f'curl -o {filename} {file_url}'

        # Execute the curl command
        subprocess.run(curl_command, check=True, shell=True)
        print(f"Downloaded '{filename}' successfully.")
    except subprocess.CalledProcessError:
        print(f"Failed to download '{filename}'. Check the URL, your internet connection and the file path")

## OpenAI

In [ ]:
#You can retrieve your API key from a file(1)
# or enter it manually(2)
#Comment this cell if you want to enter your key manually.

#(1)Retrieve the API Key from a file
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/api_key.txt", "r")
API_KEY=f.readline()
f.close()

Mounted at /content/drive


In [ ]:
try:
  import openai
except:
  !pip install openai==1.45.0
  import openai

In [ ]:
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

## Pinecone

In [ ]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 8.7 MB/s eta 0:00:00


In [ ]:
import pinecone

In [ ]:
f = open("drive/MyDrive/pinecone.txt", "r")
PINECONE_API_KEY=f.readline()
f.close()
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY


# Processing the video comments

In [ ]:
lfiles = [
    "jogging1.mp4.csv",
    "jogging2.mp4.csv",
    "skiing1.mp4.csv",
    "soccer_pass.mp4.csv",
    "soccer_player_head.mp4.csv",
    "soccer_player_running.mp4.csv",
    "surfer1.mp4.csv",
    "surfer2.mp4.csv",
    "swimming1.mp4.csv",
    "walking1.mp4.csv",
    "alpinist1.csv",
    "ball_passing_goal.mp4.csv",
    "basketball1.mp4.csv",
    "basketball2.mp4.csv",
    "basketball3.mp4.csv",
    "basketball4.mp4.csv",
    "basketball5.mp4.csv",
    "female_player_after_scoring.mp4.csv",
    "football1.mp4.csv",
    "football2.mp4.csv",
    "hockey1.mp4.csv"
]

In [ ]:
!rm -f *.csv # comments

In [ ]:
lf=len(lfiles)
print(lf)

21


In [ ]:
# Define your variables
directory = "Chapter10/comments"
for i in range(lf):
  file_name=lfiles[i]
  print(file_name)
  download(directory,file_name)

jogging1.mp4.csv
Downloaded 'jogging1.mp4.csv' successfully.
jogging2.mp4.csv
Downloaded 'jogging2.mp4.csv' successfully.
skiing1.mp4.csv
Downloaded 'skiing1.mp4.csv' successfully.
soccer_pass.mp4.csv
Downloaded 'soccer_pass.mp4.csv' successfully.
soccer_player_head.mp4.csv
Downloaded 'soccer_player_head.mp4.csv' successfully.
soccer_player_running.mp4.csv
Downloaded 'soccer_player_running.mp4.csv' successfully.
surfer1.mp4.csv
Downloaded 'surfer1.mp4.csv' successfully.
surfer2.mp4.csv
Downloaded 'surfer2.mp4.csv' successfully.
swimming1.mp4.csv
Downloaded 'swimming1.mp4.csv' successfully.
walking1.mp4.csv
Downloaded 'walking1.mp4.csv' successfully.
alpinist1.csv
Downloaded 'alpinist1.csv' successfully.
ball_passing_goal.mp4.csv
Downloaded 'ball_passing_goal.mp4.csv' successfully.
basketball1.mp4.csv
Downloaded 'basketball1.mp4.csv' successfully.
basketball2.mp4.csv
Downloaded 'basketball2.mp4.csv' successfully.
basketball3.mp4.csv
Downloaded 'basketball3.mp4.csv' successfully.
basketb

In [ ]:
directory = "/content/"

column_names = ['ID', 'FrameNumber', 'Comment', 'FileName']
df_list = []

for file in lfiles:
  file_path = os.path.join(directory, file)
  try:
    df_temp = pd.read_csv(file_path, names=column_names, header=None)
    df_list.append(df_temp)
  except pd.errors.EmptyDataError:
    print(f"No data in {file}. File is skipped.")
  except pd.errors.ParserError:
    print(f"Parsing error in {file}. Check file format. File is skipped.")
  except Exception as e:
    print(f"An error occurred with {file}: {e}")

dfl = pd.concat(df_list, ignore_index=True)

dfl

,ID,FrameNumber,Comment,FileName
0,ID,FrameNumber,Comment,FileName
1,c6850e1f-9a20-4b44-96b3-9b75ae3a603d,3,This image shows two individuals walking on a ...,jogging1.mp4
2,046b105b-4f94-420f-b66b-5af70bd02ac5,7,The image shows a forested area with tall tree...,jogging1.mp4
3,b56e343b-8427-4c74-8fd5-9d10895b9e1f,11,"In the image, there is a person who appears to...",jogging1.mp4
4,556604cd-d4d9-4476-89b4-8659b1670c74,15,The image shows a person walking on a path thr...,jogging1.mp4
...,...,...,...,...
838,5a630329-6ed2-49e1-953a-a620c718cd9d,135,"In the image, there is a group of American foo...",hockey1.mp4
839,918eda74-7234-4a5b-8262-e37e7e4ebb3d,139,This image shows a scene of American football ...,hockey1.mp4
840,0e498059-43e8-4f57-bf08-69542e060573,143,"In this image, we see a group of American foot...",hockey1.mp4
841,01355e9e-6f42-446e-9329-fb7eb9e1fe33,147,This image shows a group of people playing Ame...,hockey1.mp4


In [ ]:
number_of_lines = len(dfl)
print("Number of lines: ", number_of_lines)

Number of lines:  843


In [ ]:
df = dfl.dropna()

In [ ]:
# Count the lines
number_of_lines = len(df)
print("Number of lines: ",number_of_lines)

Number of lines:  842


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 842 entries, 0 to 842
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           842 non-null    object
 1   FrameNumber  842 non-null    object
 2   Comment      842 non-null    object
 3   FileName     842 non-null    object
dtypes: object(4)
memory usage: 32.9+ KB


# Chunking and embedding the dataset

## Chunking

In [ ]:
chunks = []

for index, row in df.iterrows():
  chunks.append(row['Comment']) # Each 'Comment' becomes its own chunk

print(f"Total num of chunks: {len(chunks)}")

Total num of chunks: 842


In [ ]:
# Print the length and content of the first 3 chunks
for i in range(3):
    print(len(chunks[i]))
    print(chunks[i])

7
Comment
463
This image shows two individuals walking on a dirt pathway through a wooded area. The foliage is dense on either side of the path, suggesting a natural, forested setting. The word "your" is superimposed on the image, which could imply that this is a frame from a video or a part of a series of images that conveys a message or a story. The scene appears calm, with no clear indications of the activity or mood of the persons, besides enjoying a walk in the woods.
436
The image shows a forested area with tall trees and dense foliage. A dirt path leads through the forest, and there is a person visible walking along the path in the distance. The word "your" is overlaid on the image, which suggests that this might be a frame from a video that is conveying a message, perhaps part of a sentence that continues beyond this single frame. It looks peaceful and suggests an outdoor or nature-themed setting.


In [ ]:
# convert the list of chunks to a DataFrame
cdf = pd.DataFrame(chunks, columns=['Chunk'])
cdf

,Chunk
0,Comment
1,This image shows two individuals walking on a ...
2,The image shows a forested area with tall tree...
3,"In the image, there is a person who appears to..."
4,The image shows a person walking on a path thr...
...,...
837,"In the image, there is a group of American foo..."
838,This image shows a scene of American football ...
839,"In this image, we see a group of American foot..."
840,This image shows a group of people playing Ame...


# Embedding

## Initialize embedding model

In [ ]:
embedding_model = "text-embedding-3-small"

client = openai.OpenAI()

# Define the function to get embeddings using the specified model
def get_embedding(text, model=embedding_model):
    # Ensure the text is a string and replace newline characters with spaces
    text = str(text).replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    embedding = response.data[0].embedding
    return embedding

In [ ]:
start_time = time.time()

# Assuming 'chunks' is already initialized as a list of text segments you want to embed
embeddings = []

for i, chunk in enumerate(chunks):
  try:
    # Get embedding for each chunk and append to the embeddings list
    embeddings.append(get_embedding(chunk))
  except Exception as e:
    print(f"Error processing chunk {i+1}: {e}")

response_time = time.time() - start_time

# Output results
print(f"All chunks processed or attempted. Total embeddings successfully created: {len(embeddings)}.")
print(f"Response Time: {response_time:.2f} seconds")

All chunks processed or attempted. Total embeddings successfully created: 842.
Response Time: 254.31 seconds


# Checking and preparing embeddings


In [ ]:
print(f"Number of embeddings: {len(embeddings)}")

Number of embeddings: 842


In [ ]:
print("First embedding overall structure:", embeddings[0])

First embedding overall structure: [0.004547559190541506, -0.004480527713894844, -0.036098238080739975, 0.016680264845490456, 0.004540503490716219, -0.0030534621328115463, -0.027038397267460823, 0.03307829052209854, -0.002220859983935952, -0.0018451306968927383, 0.01978488266468048, -0.0231858491897583, 0.00014806468971073627, 0.01193161029368639, -0.0056835669092834, -0.015339633449912071, -0.018387803807854652, -0.025034507736563683, 0.09251760691404343, 0.059834450483322144, -0.0198554415255785, 0.021125512197613716, -0.007429914548993111, -0.024879276752471924, 0.00951141957193613, -0.02610701136291027, -0.0454120896756649, 0.016045229509472847, 0.010922609828412533, -0.045778997242450714, 0.03279605135321617, -0.04047292470932007, 0.04524274542927742, -0.048262692987918854, 0.0016484461957588792, 0.017752768471837044, 0.029691435396671295, -0.012884163297712803, 0.011726987548172474, 0.009617258794605732, -0.0007889433181844652, 0.029126958921551704, 0.01677904650568962, 0.0582539

In [ ]:
first_embedding = embeddings[0]

if isinstance(first_embedding, np.ndarray):
    print("First embedding is a numpy array.")
    print("Shape:", first_embedding.shape)
    print("Data Type:", first_embedding.dtype)
    print("First few elements:", first_embedding[:10])
elif isinstance(first_embedding, list):
    print("First embedding is a list.")
    print("Length:", len(first_embedding))
    print("First few elements:", first_embedding[:10])
else:
    print("Unknown format of the embedding:", type(first_embedding))


First embedding is a list.
Length: 1536
First few elements: [0.004547559190541506, -0.004480527713894844, -0.036098238080739975, 0.016680264845490456, 0.004540503490716219, -0.0030534621328115463, -0.027038397267460823, 0.03307829052209854, -0.002220859983935952, -0.0018451306968927383]


In [ ]:
pure_embeddings = list(embeddings)

In [ ]:
r = len(pure_embeddings)
print(r)

842


## Checking chunks after embeddings

In [ ]:
num_chunks = len(chunks)
print(f"Number of chunks: {num_chunks}")

Number of chunks: 842


In [ ]:
seen_chunks = set()
duplicate_chunks = []

for chunk in chunks:
  if chunk in seen_chunks:
    duplicate_chunks.append(chunk)
  else:
    seen_chunks.add(chunk)

print(f"Number of duplicated chunks: {len(duplicate_chunks)}")

Number of duplicated chunks: 19


In [ ]:
# Check the lengths of the chunks and embeddings
num_chunks = len(chunks)
print(f"Number of chunks: {num_chunks}")
print(f"Number of embeddings: {len(pure_embeddings)}")

Number of chunks: 842
Number of embeddings: 842


In [ ]:
assert len(chunks) == len(pure_embeddings), "Mismatch between num of chunks and embeddings"

# The Pinecone index

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
from pinecone import ServerlessSpec

index_name = 'videos-sports-us' # Choose the index name of your choice
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
import time
import pinecone
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    time.sleep(1)

# Upserting

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 842 entries, 0 to 842
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           842 non-null    object
 1   FrameNumber  842 non-null    object
 2   Comment      842 non-null    object
 3   FileName     842 non-null    object
dtypes: object(4)
memory usage: 32.9+ KB


In [ ]:
len(df)

842

In [ ]:
# quality control
for index, row in df.iterrows():
    if row.isnull().any():  # Check if there is any NaN value in the row
        print(f"NaN found at index {index}:")
        print(row)  # Print the entire record

## Batch upinserting

In [ ]:
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [ ]:
# Start timing before the request
start_time = time.time()

# Prepare data for upsert
data_for_upsert = [
    {
        "id": str(df['ID'].iloc[i]),  # Use existing IDs as string
        "values": pure_embeddings[i],
        "metadata": {
            "text": df['Comment'].iloc[i],
            "frame_number": df['FrameNumber'].iloc[i],
            "file_name": df['FileName'].iloc[i]
        }
    }
    for i in range(len(df))
]

# Define a suitable batch size
batch_size = 100  # You might need to adjust this depending on the average size of your data entries

# Function to handle batching
def upsert_in_batches(data, batch_size):
    for start in range(0, len(data), batch_size):
        end = start + batch_size
        batch = data[start:end]
        try:
            index.upsert(vectors=batch)
            print(f"Upserted batch from index {start} to {end - 1}")
        except Exception as e:
            print(f"Failed to upsert batch from index {start} to {end - 1}: {e}")

# Perform upsert in batches
upsert_in_batches(data_for_upsert, batch_size)

# Measure and print response time
response_time = time.time() - start_time
print(f"Total upsertion time: {response_time:.2f} seconds")

Upserted batch from index 0 to 99
Upserted batch from index 100 to 199
Upserted batch from index 200 to 299
Upserted batch from index 300 to 399
Upserted batch from index 400 to 499
Upserted batch from index 500 to 599
Upserted batch from index 600 to 699
Upserted batch from index 700 to 799
Upserted batch from index 800 to 899
Total upsertion time: 8.69 seconds


# Statistics after upserting the dataset

In [ ]:
print(index.describe_index_stats(include_metadata=True))

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 823}},
 'total_vector_count': 823,
 'vector_type': 'dense'}


In [ ]:
# Get index information
index_info = index.describe_index_stats(include_metadata=True)

# Extract dimensions and vector count
dimensions_per_vector = index_info['dimension']
total_vectors = index_info['total_vector_count']  # or index_info['namespaces']['']['vector_count'] if using namespaces

# Output extracted information
print(f"Dimensions per vector: {dimensions_per_vector}")
print(f"Total vectors: {total_vectors}")

# Calculate storage requirements as before
bytes_per_float = 4
bytes_per_vector = dimensions_per_vector * bytes_per_float
total_bytes = bytes_per_vector * total_vectors
total_megabytes = total_bytes / (1024 ** 2)

print(f"Total estimated size of the index: {total_megabytes:.2f} MB")

Dimensions per vector: 1536
Total vectors: 823
Total estimated size of the index: 4.82 MB


# Querying vector store after upserting the dataset

In [ ]:
k = 1

query_text = "Find a basketball player that is scoring with a dunk."

In [ ]:
import time

start_time = time.time()

query_embedding = get_embedding(query_text, model=embedding_model)
query_results = index.query(vector=query_embedding, top_k=k, include_metadata=True)

print("Query Results:")
for match in query_results['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}")

    # Check if metadata is available
    if 'metadata' in match:
        metadata = match['metadata']
        text = metadata.get('text', "No text metadata available.")
        frame_number = metadata.get('frame_number', "No frame number available.")
        file_name = metadata.get('file_name', "No file name available.")

        print(f"Text: {text}")
        print(f"Frame Number: {frame_number}")
        print(f"File Name: {file_name}")
    else:
        print("No metadata available.")

# Measure response time
response_time = time.time() - start_time
print(f"Querying response time: {response_time:.2f} seconds")  # Print response time

Query Results:
ID: f104138b-0be8-4f4c-bf99-86d0eb34f7ee, Score: 0.590099454
Text: In this image, there is a person who appears to be in the process of executing a dunk in basketball. The individual is airborne, with one arm extended upwards towards the basketball hoop, holding a basketball in hand, preparing to slam it through the hoop. The word "dunk" is superimposed on the image, confirming the action taking place. The background shows clear skies and a modern building, suggesting this might be an outdoor basketball court in an urban setting. The player is wearing athletic wear and a pair of basketball shoes, suitable for the sport. The dynamic posture and the context indicate an athletic and powerful movement, typical of a basketball dunk.
Frame Number: 191
File Name: basketball3.mp4
Querying response time: 0.56 seconds


## Displaying the video

In [ ]:
print(file_name)

basketball3.mp4


In [ ]:
# downloading file from GitHub
directory = "Chapter10/videos"
filename = file_name
download(directory,file_name)

Downloaded 'basketball3.mp4' successfully.


In [ ]:
# Open the file in binary mode
def display_video(file_name):
  with open(file_name, 'rb') as file:
      video_data = file.read()

  # Encode the video file as base64
  video_url = b64encode(video_data).decode()

  # Create an HTML string with the embedded video
  html = f'''

  '''
  # Display the video
  HTML(html)
  # Return the HTML object
  return HTML(html)

display_video(file_name)